In [1]:
pwd

'/Users/Kelly/galvanize/week4/Federal-Election-Funding'

In [ ]:
'''
get file in jupyter cell
%load filename.py

save to file from jupyter cell (will overwrite)
%%writefile filename.py
'''

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [133]:
#%%writefile build_results_df.py
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# import House election results from 1976 to 2018 (CSV from Harvard Dataverse)
# then delete data prior to 1980
results_data_76to2018 = pd.read_csv('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/results-from-Dataverse/house1976-2018.csv', sep=',', header=0)
results_data_80to18 = results_data_76to2018[results_data_76to2018['year']>1979]
# import Senate election results from 1976 to 2018 (CSV from Harvard Dataverse)
# then delete data prior to 1980
senate_data = pd.read_csv('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/results-from-Dataverse/Senate/1976-2018-senate2.csv', sep=',', header=0)
senate_data_80to18 = senate_data[senate_data['year']>1979]

results_raw = pd.concat([senate_data_80to18, results_data_80to18], axis=0)

#filter out primaries (rows in which the value in column 'stage' does NOT equal 'gen')
results_gen_elec1 = results_raw[results_raw['stage']=='gen']
#filter out write ins (rows in which the boolean value in column 'writein' does NOT equal False)
results_gen_elec2 = results_gen_elec1[results_gen_elec1['writein']==False]
#filter out special (rows in which the boolean value in column 'special' does NOT equal False)
results_gen_elec3 = results_gen_elec2[results_gen_elec2['special']==False]
#add column for district string
results_gen_elec3['district_str'] = results_gen_elec3['district'].apply(lambda x: str(x) if len(str(x))>1 else '0' + str(x))
#add column for vote percentage
results_gen_elec3['vote%'] = 100 * results_gen_elec3['candidatevotes']/results_gen_elec3['totalvotes']
#add column for elec_id
results_gen_elec3['elec_id'] = results_gen_elec3['year'].astype(str) +'-'+results_gen_elec3['state_po']+'-'+results_gen_elec3['district_str']


#extract last name from candidate column (potential for error) & make new column
surname_series2 = results_gen_elec3['candidate'].apply(lambda x: x.split(' ')[-1].upper() if x.split(' ')[-1] not in nix_name else x.split(' ')[-2].upper())
#strip trailing commas 
results_gen_elec3['last_name'] = surname_series2.str.strip(',')
#look for faulty entries, particularly among frequently occurring names  
nix_name = ['Sr', 'Sr.', 'Jr', 'Jr.', 'III', 'II']
surname_series = results_gen_elec3['candidate'].apply(lambda x: x.split(' ')[-1] if x.split(' ')[-1] not in nix_name else x.split(' ')[-2])
# make list of all names that occur and remove bad entries ['Other', 'Vote/Scattering', 'scatter', 'Vote', ' ']
# *** ' ' = likely due to flaw in code
surname_lst = list(surname_series.value_counts().index)
last_name_list = [x.upper() for x in (surname_lst[3:7]+surname_lst[8:139] + surname_lst[140:])]
last_name_list
results_gen_elec3[results_gen_elec3['last_name'].isin(last_name_list)]
# make ELEC-CAND column (first edit elec_id to match formatting of finance_df)
results_gen_elec3['elec_id'] = results_gen_elec3['elec_id'].str.replace('statewide', '00')
results_gen_elec3['elec-cand'] = results_gen_elec3['elec_id'] + '-' + results_gen_elec3['last_name']
# remove duplicates from ELEC-CAND
repeats = results_gen_elec3['elec-cand'].value_counts()
repeats_df = pd.DataFrame()
repeats_df['filtered'] = repeats[repeats > 1]
mystery_repeats = results_gen_elec3[results_gen_elec3['elec-cand'].isin(repeats_df.reset_index()['index'])]
results_GE = results_gen_elec3[~results_gen_elec3['elec-cand'].isin(repeats_df.reset_index()['index'])]
# ready for merge? 
results_GE

/Users/Kelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/Kelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Kelly/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

,candidate,candidatevotes,district,mode,office,party,runoff,special,stage,state,...,totalvotes,unofficial,version,writein,year,district_str,vote%,elec_id,last_name,elec-cand
270,Jim Folsom Jr.,610175,statewide,total,US Senate,democrat,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,47.053920,1980-AL-00,FOLSOM,1980-AL-00-FOLSOM
271,Jeremiah Denton,650362,statewide,total,US Senate,republican,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,50.152958,1980-AL-00,DENTON,1980-AL-00-DENTON
272,Sallie M. Hadnott,2973,statewide,total,US Senate,national democratic party of alabama,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,0.229264,1980-AL-00,HADNOTT,1980-AL-00-HADNOTT
273,William A. Crew,13098,statewide,total,US Senate,libertarian,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,1.010058,1980-AL-00,CREW,1980-AL-00-CREW
274,Michael R.A. Erdey,15989,statewide,total,US Senate,conservative,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,1.232999,1980-AL-00,ERDEY,1980-AL-00-ERDEY
275,Jim Partain,2649,statewide,total,US Senate,statesman,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,0.204279,1980-AL-00,PARTAIN,1980-AL-00-PARTAIN
276,Mohammed Oliver,1511,statewide,total,US Senate,socialist workers,NaN,False,gen,Alabama,...,1296757,False,20171011.0,False,1980,statewide,0.116521,1980-AL-00,OLIVER,1980-AL-00-OLIVER
277,Clark S. Gruening,72007,statewide,total,US Senate,democrat,NaN,False,gen,Alaska,...,156762,False,20171011.0,False,1980,statewide,45.933964,1980-AK-00,GRUENING,1980-AK-00-GRUENING
279,Frank H. Murkowski,84159,statewide,total,US Senate,republican,NaN,False,gen,Alaska,...,156762,False,20171011.0,False,1980,statewide,53.685842,1980-AK-00,MURKOWSKI,1980-AK-00-MURKOWSKI
281,Josefina Otero,3266,statewide,total,US Senate,socialist workers,NaN,False,gen,Arizona,...,874238,False,20171011.0,False,1980,statewide,0.373582,1980-AZ-00,OTERO,1980-AZ-00-OTERO
